# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "Weather_database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Beyneu,KZ,45.3167,55.2000,31.30,88,5,8.93,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,48.31,66,75,23.02,broken clouds
2,2,Conakry,GN,9.5380,-13.6773,80.58,77,26,10.63,scattered clouds
3,3,Batagay-Alyta,RU,67.8006,130.4114,-33.30,89,57,3.56,broken clouds
4,4,Megion,RU,61.0296,76.1136,1.42,96,44,9.66,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Puerto Ayora,EC,-0.7393,-90.3518,70.56,95,89,7.20,overcast clouds
14,14,Acari,PE,-15.4311,-74.6158,73.11,83,88,5.14,overcast clouds
17,17,Panaba,MX,21.2833,-88.2667,79.56,66,15,14.58,few clouds
19,19,Mar Del Plata,AR,-38.0023,-57.5575,75.13,58,75,18.41,thunderstorm
23,23,Grand Gaube,MU,-20.0064,57.6608,73.38,89,25,1.99,light rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Puerto Ayora,EC,-0.7393,-90.3518,70.56,95,89,7.20,overcast clouds
14,14,Acari,PE,-15.4311,-74.6158,73.11,83,88,5.14,overcast clouds
17,17,Panaba,MX,21.2833,-88.2667,79.56,66,15,14.58,few clouds
19,19,Mar Del Plata,AR,-38.0023,-57.5575,75.13,58,75,18.41,thunderstorm
23,23,Grand Gaube,MU,-20.0064,57.6608,73.38,89,25,1.99,light rain
...,...,...,...,...,...,...,...,...,...,...
655,655,Carutapera,BR,-1.1950,-46.0200,77.00,85,92,6.11,overcast clouds
656,656,Kuching,MY,1.5500,110.3333,79.03,94,20,2.30,mist
664,664,Eyl,SO,7.9803,49.8164,71.44,70,4,11.10,clear sky
666,666,Flinders,AU,-34.5833,150.8552,72.18,43,11,8.99,few clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat","Lng"]].copy()


# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
11,Puerto Ayora,EC,70.56,overcast clouds,-0.7393,-90.3518
14,Acari,PE,73.11,overcast clouds,-15.4311,-74.6158
17,Panaba,MX,79.56,few clouds,21.2833,-88.2667
19,Mar Del Plata,AR,75.13,thunderstorm,-38.0023,-57.5575
23,Grand Gaube,MU,73.38,light rain,-20.0064,57.6608
...,...,...,...,...,...,...
655,Carutapera,BR,77.00,overcast clouds,-1.1950,-46.0200
656,Kuching,MY,79.03,mist,1.5500,110.3333
664,Eyl,SO,71.44,clear sky,7.9803,49.8164
666,Flinders,AU,72.18,few clouds,-34.5833,150.8552


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [9]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Puerto Ayora,EC,70.56,overcast clouds,-0.7393,-90.3518,
14,Acari,PE,73.11,overcast clouds,-15.4311,-74.6158,
17,Panaba,MX,79.56,few clouds,21.2833,-88.2667,
19,Mar Del Plata,AR,75.13,thunderstorm,-38.0023,-57.5575,
23,Grand Gaube,MU,73.38,light rain,-20.0064,57.6608,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [15]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [16]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Puerto Ayora,EC,70.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Acari,PE,73.11,overcast clouds,-15.4311,-74.6158,HOSPEDAJE XIMENA
17,Panaba,MX,79.56,few clouds,21.2833,-88.2667,Hotel Don Fernando
19,Mar Del Plata,AR,75.13,thunderstorm,-38.0023,-57.5575,Gran Hotel Mar del Plata
23,Grand Gaube,MU,73.38,light rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
...,...,...,...,...,...,...,...
655,Carutapera,BR,77.00,overcast clouds,-1.1950,-46.0200,Lidera
656,Kuching,MY,79.03,mist,1.5500,110.3333,Hilton Kuching
664,Eyl,SO,71.44,clear sky,7.9803,49.8164,EYL AQUE SYSTEM
666,Flinders,AU,72.18,few clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [24]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Puerto Ayora,EC,70.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Acari,PE,73.11,overcast clouds,-15.4311,-74.6158,HOSPEDAJE XIMENA
17,Panaba,MX,79.56,few clouds,21.2833,-88.2667,Hotel Don Fernando
19,Mar Del Plata,AR,75.13,thunderstorm,-38.0023,-57.5575,Gran Hotel Mar del Plata
23,Grand Gaube,MU,73.38,light rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
...,...,...,...,...,...,...,...
655,Carutapera,BR,77.00,overcast clouds,-1.1950,-46.0200,Lidera
656,Kuching,MY,79.03,mist,1.5500,110.3333,Hilton Kuching
664,Eyl,SO,71.44,clear sky,7.9803,49.8164,EYL AQUE SYSTEM
666,Flinders,AU,72.18,few clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [25]:
# Set the file name.
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [27]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [35]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n<dt>City</dt><dd>Puerto Ayora</dd>\n<dt>Country</dt><dd>EC</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 70.56 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>HOSPEDAJE XIMENA</dd>\n<dt>City</dt><dd>Acari</dd>\n<dt>Country</dt><dd>PE</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 73.11 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Don Fernando</dd>\n<dt>City</dt><dd>Panaba</dd>\n<dt>Country</dt><dd>MX</dd>\n<dt>Current Weather</dt><dd>few clouds and 79.56 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Gran Hotel Mar del Plata</dd>\n<dt>City</dt><dd>Mar Del Plata</dd>\n<dt>Country</dt><dd>AR</dd>\n<dt>Current Weather</dt><dd>thunderstorm and 75.13 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Veranda Paul et Virginie Hotel & Spa</dd>\n<dt>City</dt><dd>Grand Gaube</dd>\n<dt>Country</dt><dd>MU</dd>\n<dt>Current Weather</dt><dd>light rain and 73.38 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hot

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [36]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
11,-0.7393,-90.3518
14,-15.4311,-74.6158
17,21.2833,-88.2667
19,-38.0023,-57.5575
23,-20.0064,57.6608
26,-0.9000,-89.6000
31,-3.3244,114.5910
32,-33.0153,27.9116
33,-23.1203,-134.9692
39,-23.3500,47.6000


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [37]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
              
# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))